<br>
<br>

![iteso](https://upload.wikimedia.org/wikipedia/en/5/5f/Western_Institute_of_Technology_and_Higher_Education_logo.png)

###  InstitutoTecnológico y de Estudios Superiores de Occidente ###
###  Maestría Ciencia de Datos  ###
###  Optimización Convexa ###
###  HW 04: Optimización restringida y regularización (problema 2) ###

<br>
<br>

* * *

Estudiante: Daniel Nuño <br>
Profesor: Dr. Juan Diego Sanchez Torres <br>
Fecha entrega: 16 de febrero, 2022 <br>

* * *

<br>
<br>

#### Introducción ####

The purpose of this last activity is to solve a problem similar to those encountered in practice, using a reasonably sophisticated code structure appropriate to the solution of the problem. Also, use is made of symbolic transformers for the resolution of a regression problem. It is demonstrated that the use of new inputs can improve the model’s predictive capacity.

#### Probelma 2 ####

Read and reproduce the example about the Boston housing dataset given in [Gplearn: Symbolic Transformer](https://gplearn.readthedocs.io/en/stable/examples.html#symbolic-transformer). Then, explain how the symbolic transformer method helps to improve the regression’s performance. Upload your results to Github in the form of a Jupyter notebook, then make it interactive using Binder, hence submit your results through both links. The use of Google Colab is highly recommended.


El ejemplo demuestra cómo crear nuevas variables no lineares usando `SymbolicTransformer` de la paquetería `gplearn`.


In [36]:
%matplotlib inline
from gplearn.genetic import SymbolicTransformer
from sklearn.utils import check_random_state
from sklearn.datasets import load_boston
import numpy as np

In [44]:
rng = check_random_state(0)
boston = load_boston()
perm = rng.permutation(boston.target.size)
boston.data = boston.data[perm]
boston.target = boston.target[perm]

Usando `Ridge` de `sklearn` se utilizará como base de comparación. Entrenas con las primeras 300 observaciones de boston usando Ridge y ves como desempeña con las finales 200 observaciones. El benchmark a batir es usando simplemente Ridge.

La precisión que obtuvimos es 0.7593.

In [38]:
from sklearn.linear_model import Ridge
est = Ridge()
est.fit(boston.data[:300, :], boston.target[:300])
print(est.score(boston.data[300:, :], boston.target[300:]))

0.7593194530498838


En la siguiente celda, se entrena el transformador sobre las primeras 300 observaciones para generar nuevas variables.

Un trasformador simbólico de Programación Genética es un transformador supervisado que comienza construyendo una población de fórmulas aleatorias ingenuas para representar una relación. Las fórmulas se representan como estructuras de forma de árbol con funciones matemáticas que se aplican recursivamente a variables y constantes. Luego, cada generación sucesiva de programas se desarrolla a partir de la anterior, seleccionando a los individuaos más aptos de la población para que se sometan a operaciones genéticas como el cruzamiento, la mutación o a la reproducción. La población final se busca los individuos más aptos con la menor correlación entre sí.

Los argumentos que toma son los siguientes:<br>
- `population_size:` es un número entero de programas en cada generación.<br>
- `generations:` es el número de generaciones a evolucionar.<br>
- `hall_of_fame:` es el número de los programas más aptos para comparar cuando se encuentren los individuos menos correlacionados para los *n_components*. Si es marcado como *None*, entonces toda la generación final sera usada.<br>
- `n_components:` es el número de mejores programas para regresar después de buscar en *hall_of_fame* para los individuos menos correlacionados. Si *None* es indicado, entonces todo el *hall_of_fame* sera usado.<br>
- `function_set:` las funciones que se utilizarán al crear y desarrollar programas. Este iterable puede incluir *strings* para indicar funciones individuales como se describe a continuación, o también puede incluir sus propias funciones creadas con la `make_function` del módulo `funciones`. En este ejemplo se utiliza suma, resta, multiplicación, división, raíz cuadrada, logaritmo natural, valor absoluto, negativos, inversa, máximos y mínimos.<br>
- `parsimony_coefficient:` decimal o 'auto'. Esta constante penaliza los grandes programas ajustando su aptitud para que sean menos favorables para la selección. Los valores más grandes penalizan más al programa que puede controlar el fenómeno conocido como "bloat". Bloat es cuando la evolución aumenta el tamaño de los programas sin un aumento significativo en la aptitud, lo que es costoso para el tiempo de cálculo y hace que el resultado final sea menos comprensible. Es posible que este parámetro deba ajustarse en ejecuciones sucesivas.<br>
- `max_samples:` es un número decimal que indica la fracción de muestras a sacar a partir de X a evaluar con cada programa.<br>
- `random_state:` Si es un entero, random_state es la semilla utilizada por el generador de números aleatorios; Si es una instancia de RandomState, random_state es el generador de números aleatorios; Si es Ninguno, el generador de números aleatorios es la instancia de RandomState utilizada por np.random. <br>
- `verbose:` sirve para imprimir los resultados del algoritmo al final de cada generación.<br>

Así que ahora entrenaremos a nuestro transformador con las mismas primeras 300 muestras para generar características nuevas. Usemos una gran población de 2000 individuos durante 20 generaciones. Seleccionaremos los mejores 100 de estos para el salón de la fama y luego usaremos los 10 menos correlacionados como nuestras nuevas variables. Un poco de avaricia debería controlar el bloat, pero dejaremos el resto de las opciones de evolución en sus valores predeterminados. La métrica predeterminada = 'pearson' es apropiada aquí ya que estamos usando un modelo lineal como estimador. Si fuéramos a utilizar un estimador basado en árboles, también sería interesante probar la correlación de Spearman:

In [39]:
function_set = ['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                'abs', 'neg', 'inv', 'max', 'min']
gp = SymbolicTransformer(generations=20, population_size=2000,
                         hall_of_fame=100, n_components=10,
                         function_set=function_set,
                         parsimony_coefficient=0.0005,
                         max_samples=0.9, verbose=1,
                         random_state=0)
gp.fit(boston.data[:300, :], boston.target[:300])

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.04         0.339876        6         0.822502         0.675124     24.78s
   1     6.91         0.593562        7         0.836993         0.602468     22.55s
   2     5.07         0.730093        8          0.84063         0.704017     21.91s
   3     5.22         0.735525        5         0.847019         0.628351     21.53s
   4     6.24         0.734679       10         0.856612         0.565138     19.41s
   5     8.23         0.721433       18          0.85677         0.728095     17.97s
   6    10.20         0.717937       14         0.875233         0.619693     19.22s
   7    11.84         0.720667       14         0.875927         0.609363     16.53s
   8    12.56         0.733019       27         0.881705         0.390121  

SymbolicTransformer(function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                                  'abs', 'neg', 'inv', 'max', 'min'],
                    max_samples=0.9, parsimony_coefficient=0.0005,
                    population_size=2000, random_state=0, verbose=1)

Luego aplicaremos nuestro transformador entrenado a todo el conjunto de datos de Boston (recuerde, todavía no ha visto las 200 muestras finales) y lo concatenamos con los datos originales:

In [40]:
gp_features = gp.transform(boston.data)
new_boston = np.hstack((boston.data, gp_features))

Ahora entrenamos el regresor de Ridge en las primeras 300 muestras del conjunto de datos transformado y vemos cómo se comporta nuevamente en las 200 finales:

In [41]:
est = Ridge()
est.fit(new_boston[:300, :], boston.target[:300])
print(est.score(new_boston[300:, :], boston.target[300:]))

0.8418372105182075


El resultado son 20 épocas o generaciones.
- average_length: es el promedio de longitud de la generación.
- average_fitness: es el promedio de aptitud de la generación.
- best_length: la longitud del mejor programa en la generación.
- best_fitness: la aptitud del mejor programa en la generación.
- best_oob_fitness: *out of the bag* aptitud del mejor programa en la generación.

La precisión con esta estimación es 0.84.

Hemos mejorado la puntuación R2 por un margen significativo. Parece que el modelo lineal pudo aprovechar algunas características no lineales nuevas para ajustar los datos aún mejor.

Las nuevas características son, como lo habíamos mencionado 10. El total de columnas en *new_boston* son 23.

In [42]:
new_boston.shape

(506, 23)

En conclusión, este algoritmo mejoró el resultado de la regresión lineal por que generó nuevas variables, de una forma *aleatoria e ingenua* evolutivamente. Dichas variables son las que aportaban mejores resultados son las que tienen menor correlacion entre si.